
|  |
| ------------------------------------------------------- | 
| ![Tremplin des sciences](images/tremplinColorSmall.png) | 

Cahier d'exercices pour l'enseignement du changement climat climatique ou l'apprentissage de programmation issu de la collection "Climat et météo tremplin pour l'enseignement des sciences" (PIA IFÉ ENS de Lyon - Météofrance ENM Toulouse). Le dispositif clef en main repose sur l'utilisation d'une RaspberryPi chargée avec le système d'exploitation Debian enrichi, produit par le projet. Les sources et les exécutables sont accessibles dans [l'espace collaboratif du pojet à l'IFÉ ENS de Lyon](https://contrib-tremplin.ens-lyon.fr/) et une copie se trouve dans [l'espace collaboratif de la forge github](https://github.com/g-vidal/CahierDeProgrammes); plus d'information sur les [blogs d'accompagnement](http://blog.climatetmeteo.fr/GerardVidal/) systèmes d'exploitation sur [la page des OS  de Raspberries Pi](http://mediaserv.climatetmeteo.fr/images/RaspBerry/DebianStretchPi3/).  Toutes les ressources issues du projet sont fournies sous licence [Creative Commons](https://creativecommons.org/licenses/by-nc/4.0/) ou sous les licences libres d'origine des outils utilisés. Les ressources  du projet peuvent être utilisées dans tout autre environnement compatible. Les données en ligne sont accessibles sur le [serveur de données géolocalisées](http://geoloc-tremplin.ens-lyon.fr/climato-data/) `opendap` du projet tremplin.

![licence : Creative Commons](images/Licence.jpg) 

Auteur : G. Vidal

------------------------------------------------------------

# Une approche du changement climatique à Lyon : Conversion d'un jeu de données extrait de DRIAS pour le serveur du projet Tremplin

Ce cahier d'exercices `ipython` propose une méthode de conversion d'un jeu de données issues des simulations climatiques de Météofrance. Le résultat de trois séries de modélisations du CNRM  RCP 2.6 - 4.5 - 8.5 est utilisé ici.  Les fichiers commandés sur le site  [DRIAS](http://www.drias-climat.fr/), ne peuvent être (encore) directement servis par le serveur du projet (limitation du logiciel OpenDap). Ce cahier permet de les convertir pour permettre leur exploitation en ligne à l'aide des autres cahiers de programmes proposés par le projet sur ce site. 

Les programmes ci-dessous **ne peuvent pas être complètement génériques** et une interface graphique permettant de fournir interactivement les paramètres Ad-Hoc n'a pas été développée. les extractions de [DRIAS](http://www.drias-climat.fr/) sont très cohérentes  et il suffit pour réutiliser ce cahier d'exercices :
    1. De repérer les noms des répertoires où se trouvent les fichiers originaux de Drias, et le répertoire où seront sauvegardés les fichiers transformés et de les remplacer.
    2.  De repérer les noms des fichiers à traiter et de les remplacer
    3. De repérer le nom de la variable principale et de le remplacer si besoin (À cette fin un bloc de programmes permettant d'imprimer  les paramètres de chaque fichier est proposé il suffit de décommenter les lignes  réalisant une impression).

Ce cahier manipule des données multidimensionnelles et doit être réservé à des étudiants avancés si on souhaite l'utiliser en classe. Les données d'un carré 10 x 10 noeuds de la grille ALADIN centré sur la ville de Lyon. Le format d'entrée et de sortie est celui utilisé par les labos de climatologie, d'océanographie : `netCDF`.  Cet outil requiert l'installation des outils `netCDF4` et `numpy`

## Préparation de l'environnement et ouverture du fichier de données

Importer d'abord le module `netcdf4` et `numpy`, attention les majuscules sont impératives pour le nom `netCDF4`. Ces deux modules permettent de traiter  les fichiers multidimensionnels au format netCDF utilisés dans le monde de la météorologie et de l'océanographie principalement.

In [1]:
import netCDF4 as nc
import numpy as np
from datetime import datetime
from array import array
import sys, datetime, os

## Traitement des données par variables

Importation des données depuis le fichier obtenu auprès du site [DRIAS](https://drias-prod.meteo.fr/okapi/accueil/okapiWebDrias/index.jsp) pour la région lyonnaise, construction d'un jeu de données en convertissant la totalité des variables codées en 64 bits vers un codage sur 32 bits pour que le serveur opendap les gère convenablement.

### Hygrométrie

Détection des variables (facultatif), pour obtenir les informations décommenter les lignes d'impression des valeurs. 

In [2]:
for name in ['huss_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc',
             'huss_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',
             'huss_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',
             'huss_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc'
            ] :
    namein = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/' + name
    nameout = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/' + name
#    print (namein, '\n',  nameout,'\n')
    origDrias = nc.Dataset(namein)
#    print('Dimensions disponibles :', origDrias.dimensions.keys(),'\n') # get all variable names
#    for var in origDrias.dimensions.keys() :
#        print ('\t', origDrias.dimensions[var],
#               '\t', origDrias.dimensions[var].name,
#               '\t', origDrias.dimensions[var].size)
#    print('\nVariables disponibles :', origDrias.variables.keys(),'\n') # get all variable names
#    for var in origDrias.variables.keys() :
#        print ('\t', var,
#               '\t', origDrias.variables[var].dimensions, 
#               '\t', origDrias.variables[var].shape, 
#               '\t', origDrias.variables[var].dtype)

In [3]:
for name in ['huss_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc',
             'huss_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',
             'huss_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',
             'huss_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc'
            ] :
    namein = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/' + name
    nameout = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/' + name
    print (namein, '\n',  nameout,'\n')
    origDrias = nc.Dataset(namein)
    try: os.remove(nameout)  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
    except OSError : pass
    convertedSet = nc.Dataset(nameout, mode='w', format='NETCDF4') 
    for var in origDrias.dimensions.keys() :
        thisDim = origDrias.dimensions[var]
        if (thisDim.name == 'time') :
            convertedSet.createDimension(thisDim.name)
        else :
            convertedSet.createDimension(thisDim.name, thisDim.size)     # latitude axis
#
# variables detected in the original file
#
    huss = convertedSet.createVariable('huss', 'f4', ('time','j','i'),fill_value=1.e+20)
    huss.units = 'l'
    huss.long_name = 'Near-Surface Specific Humidity'
    huss.standard_name = 'specific_humidity'
    huss._CoordinateAxisType = 'lon lat'
    huss[:,:,:] = origDrias.variables['huss'][:,:,:] 
#    print('\t\t huss : ',huss)
    i = convertedSet.createVariable('i', 'f4', ('i',))
    i.long_name = 'cell index along first dimension'
    i[:] = origDrias.variables['i'][:]
#    print('\t i :',i)
#    print('\t',tMax_26_Lyon.variables['i'][:])
    j = convertedSet.createVariable('j', 'f4', ('j',))
    j.long_name = 'cell index along second dimension'
    j[:] = origDrias.variables['j'][:]
#    print('\t j :',j)
    lat = convertedSet.createVariable('lat', 'f4', ('j','i'))
    lat.units = 'degrees_north'
    lat.long_name = 'latitude'
    lat.standard_name = 'latitude'
    lat._CoordinateAxisType = 'Lat'
    lat[:,:]  = origDrias.variables['lat'][:,:] 
#    print('\t lat :',lat)
    lon = convertedSet.createVariable('lon', 'f4', ('j','i',))
    lon.units = 'degrees_east'
    lon.long_name = 'longitude'
    lon.standard_name = 'longitude'
    lon._CoordinateAxisType = 'Lon'
    lon[:,:] = origDrias.variables['lon'][:,:]
#    print('\t lon :',lon)
    time = convertedSet.createVariable('time', 'f4', ('time',))
    time.units = 'hours since 1949-12-01 00:00:00'
    time.long_name = 'time'
    time.standard_name = 'time'
    time[:] = origDrias.variables['time'][:]
#    print('\t time :',time)
    x = convertedSet.createVariable('x', 'i4', ('i',))
    x.units = 'm'
    x.long_name = 'cell index along first dimension'
    x.standard_name = 'projection_x_coordinate'
    x[:] = origDrias.variables['x'][:]
#    print('\t x :',x)
    y = convertedSet.createVariable('y', 'i4', ('j',))
    y.units = 'm'
    y.long_name = 'cell index along second dimension'
    y.standard_name = 'projection_x_coordinate'
    y[:] = origDrias.variables['y'][:]
#    print('\t y :',y)
    convertedSet.Conventions = origDrias.Conventions ;
    convertedSet.institution = origDrias.institution ;
    convertedSet.institute_id = origDrias.institute_id ;
    convertedSet.project_id =origDrias.project_id ;
    convertedSet.model_id = origDrias.model_id ;
    convertedSet.frequency = origDrias.frequency ;
    convertedSet.product = origDrias.product ;
    convertedSet.contact = origDrias.contact ;
    convertedSet.references = origDrias.references ;
    convertedSet.creation_date = origDrias.creation_date ;
    convertedSet.NCO = origDrias.NCO ;
    convertedSet.driving_experiment_name = origDrias.driving_experiment_name ;
    convertedSet.experiment = origDrias.experiment ;
    convertedSet.experiment_id = origDrias.experiment_id ;
    convertedSet.model = origDrias.model ;
    convertedSet.comment = origDrias.comment ;
    convertedSet.history = origDrias.history ;
    convertedSet.nco_openmp_thread_number = origDrias.nco_openmp_thread_number ;
#    for var in convertLyonTempYearMonth.variables.keys() :
#        print ('\t', var,
#               '\t', convertLyonTempYearMonth.variables[var].dimensions, 
#               '\t', convertLyonTempYearMonth.variables[var].shape, 
#               '\t', convertLyonTempYearMonth.variables[var].dtype)
    convertedSet.close()

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/huss_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/huss_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc 

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/huss_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/huss_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc 

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/huss_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/huss_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc 

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/huss_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/huss_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc 



### Précipitations neigeuses

détection des variables (facultatif)

In [4]:
for name in ['prsnls_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc',
             'prsnls_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',
             'prsnls_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',
             'prsnls_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc'
            ] :
    namein = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/' + name
    nameout = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/' + name
#    print (namein, '\n',  nameout,'\n')
    origDrias = nc.Dataset(namein)
#    print('Dimensions disponibles :', origDrias.dimensions.keys(),'\n') # get all variable names
#    for var in origDrias.dimensions.keys() :
#        print ('\t', origDrias.dimensions[var],
#               '\t', origDrias.dimensions[var].name,
#               '\t', origDrias.dimensions[var].size)
#    print('\nVariables disponibles :', origDrias.variables.keys(),'\n') # get all variable names
#    for var in origDrias.variables.keys() :
#        print ('\t', var,
#               '\t', origDrias.variables[var].dimensions, 
#               '\t', origDrias.variables[var].shape, 
#               '\t', origDrias.variables[var].dtype)

In [5]:
for name in ['prsnls_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc',
             'prsnls_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',
             'prsnls_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',
             'prsnls_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc'
            ] :
    namein = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/' + name
    nameout = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/' + name
    print (namein, '\n',  nameout,'\n')
    origDrias = nc.Dataset(namein)
    try: os.remove(nameout)  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
    except OSError : pass
    convertedSet = nc.Dataset(nameout, mode='w', format='NETCDF4') 
    for var in origDrias.dimensions.keys() :
        thisDim = origDrias.dimensions[var]
        if (thisDim.name == 'time') :
            convertedSet.createDimension(thisDim.name)
        else :
            convertedSet.createDimension(thisDim.name, thisDim.size)     # latitude axis
#
# variables detected in the original file
#
    prsnls = convertedSet.createVariable('prsnls', 'f4', ('time','j','i'),fill_value=1.e+20)
    prsnls.units = 'l'
    prsnls.long_name = 'Near-Surface Specific Humidity'
    prsnls.standard_name = 'specific_humidity'
    prsnls._CoordinateAxisType = 'lon lat'
    prsnls[:,:,:] = origDrias.variables['prsnls'][:,:,:] 
#    print('\t\t huss : ',huss)
    i = convertedSet.createVariable('i', 'f4', ('i',))
    i.long_name = 'cell index along first dimension'
    i[:] = origDrias.variables['i'][:]
#    print('\t i :',i)
#    print('\t',tMax_26_Lyon.variables['i'][:])
    j = convertedSet.createVariable('j', 'f4', ('j',))
    j.long_name = 'cell index along second dimension'
    j[:] = origDrias.variables['j'][:]
#    print('\t j :',j)
    lat = convertedSet.createVariable('lat', 'f4', ('j','i'))
    lat.units = 'degrees_north'
    lat.long_name = 'latitude'
    lat.standard_name = 'latitude'
    lat._CoordinateAxisType = 'Lat'
    lat[:,:]  = origDrias.variables['lat'][:,:] 
#    print('\t lat :',lat)
    lon = convertedSet.createVariable('lon', 'f4', ('j','i',))
    lon.units = 'degrees_east'
    lon.long_name = 'longitude'
    lon.standard_name = 'longitude'
    lon._CoordinateAxisType = 'Lon'
    lon[:,:] = origDrias.variables['lon'][:,:]
#    print('\t lon :',lon)
    time = convertedSet.createVariable('time', 'f4', ('time',))
    time.units = 'hours since 1949-12-01 00:00:00'
    time.long_name = 'time'
    time.standard_name = 'time'
    time[:] = origDrias.variables['time'][:]
#    print('\t time :',time)
    x = convertedSet.createVariable('x', 'i4', ('i',))
    x.units = 'm'
    x.long_name = 'cell index along first dimension'
    x.standard_name = 'projection_x_coordinate'
    x[:] = origDrias.variables['x'][:]
#    print('\t x :',x)
    y = convertedSet.createVariable('y', 'i4', ('j',))
    y.units = 'm'
    y.long_name = 'cell index along second dimension'
    y.standard_name = 'projection_x_coordinate'
    y[:] = origDrias.variables['y'][:]
#    print('\t y :',y)
    convertedSet.Conventions = origDrias.Conventions ;
    convertedSet.institution = origDrias.institution ;
    convertedSet.institute_id = origDrias.institute_id ;
    convertedSet.project_id =origDrias.project_id ;
    convertedSet.model_id = origDrias.model_id ;
    convertedSet.frequency = origDrias.frequency ;
    convertedSet.product = origDrias.product ;
    convertedSet.contact = origDrias.contact ;
    convertedSet.references = origDrias.references ;
    convertedSet.creation_date = origDrias.creation_date ;
    convertedSet.NCO = origDrias.NCO ;
    convertedSet.driving_experiment_name = origDrias.driving_experiment_name ;
    convertedSet.experiment = origDrias.experiment ;
    convertedSet.experiment_id = origDrias.experiment_id ;
    convertedSet.model = origDrias.model ;
    convertedSet.comment = origDrias.comment ;
    convertedSet.history = origDrias.history ;
    convertedSet.nco_openmp_thread_number = origDrias.nco_openmp_thread_number ;
#    for var in convertLyonTempYearMonth.variables.keys() :
#        print ('\t', var,
#               '\t', convertLyonTempYearMonth.variables[var].dimensions, 
#               '\t', convertLyonTempYearMonth.variables[var].shape, 
#               '\t', convertLyonTempYearMonth.variables[var].dtype)
    convertedSet.close()

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/prsnls_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/prsnls_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc 

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/prsnls_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/prsnls_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc 

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/prsnls_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/prsnls_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc 

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/prsnls_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/prsnls_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc 



### précipitations

détection des variables (facultatif)

In [6]:
for name in ['rstr_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc',
             'rstr_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',
             'rstr_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',
             'rstr_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc'
            ] :
    namein = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/' + name
    nameout = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/' + name
#    print (namein, '\n',  nameout,'\n')
    origDrias = nc.Dataset(namein)
#    print('Dimensions disponibles :', origDrias.dimensions.keys(),'\n') # get all variable names
#    for var in origDrias.dimensions.keys() :
#        print ('\t', origDrias.dimensions[var],
#               '\t', origDrias.dimensions[var].name,
#               '\t', origDrias.dimensions[var].size)
#    print('\nVariables disponibles :', origDrias.variables.keys(),'\n') # get all variable names
#    for var in origDrias.variables.keys() :
#        print ('\t', var,
#               '\t', origDrias.variables[var].dimensions, 
#               '\t', origDrias.variables[var].shape, 
#               '\t', origDrias.variables[var].dtype)

In [7]:
for name in ['rstr_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc',
             'rstr_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',
             'rstr_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',
             'rstr_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc'
            ] :
    namein = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/' + name
    nameout = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/' + name
    print (namein, '\n',  nameout,'\n')
    origDrias = nc.Dataset(namein)
    try: os.remove(nameout)  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
    except OSError : pass
    convertedSet = nc.Dataset(nameout, mode='w', format='NETCDF4') 
    for var in origDrias.dimensions.keys() :
        thisDim = origDrias.dimensions[var]
        if (thisDim.name == 'time') :
            convertedSet.createDimension(thisDim.name)
        else :
            convertedSet.createDimension(thisDim.name, thisDim.size)     # latitude axis
#
# variables detected in the original file
#
    rstr = convertedSet.createVariable('rstr', 'f4', ('time','j','i'),fill_value=1.e+20)
    rstr.units = 'l'
    rstr.long_name = 'Near-Surface Specific Humidity'
    rstr.standard_name = 'specific_humidity'
    rstr._CoordinateAxisType = 'lon lat'
    rstr[:,:,:] = origDrias.variables['rstr'][:,:,:] 
#    print('\t\t huss : ',huss)
    i = convertedSet.createVariable('i', 'f4', ('i',))
    i.long_name = 'cell index along first dimension'
    i[:] = origDrias.variables['i'][:]
#    print('\t i :',i)
#    print('\t',tMax_26_Lyon.variables['i'][:])
    j = convertedSet.createVariable('j', 'f4', ('j',))
    j.long_name = 'cell index along second dimension'
    j[:] = origDrias.variables['j'][:]
#    print('\t j :',j)
    lat = convertedSet.createVariable('lat', 'f4', ('j','i'))
    lat.units = 'degrees_north'
    lat.long_name = 'latitude'
    lat.standard_name = 'latitude'
    lat._CoordinateAxisType = 'Lat'
    lat[:,:]  = origDrias.variables['lat'][:,:] 
#    print('\t lat :',lat)
    lon = convertedSet.createVariable('lon', 'f4', ('j','i',))
    lon.units = 'degrees_east'
    lon.long_name = 'longitude'
    lon.standard_name = 'longitude'
    lon._CoordinateAxisType = 'Lon'
    lon[:,:] = origDrias.variables['lon'][:,:]
#    print('\t lon :',lon)
    time = convertedSet.createVariable('time', 'f4', ('time',))
    time.units = 'hours since 1949-12-01 00:00:00'
    time.long_name = 'time'
    time.standard_name = 'time'
    time[:] = origDrias.variables['time'][:]
#    print('\t time :',time)
    x = convertedSet.createVariable('x', 'i4', ('i',))
    x.units = 'm'
    x.long_name = 'cell index along first dimension'
    x.standard_name = 'projection_x_coordinate'
    x[:] = origDrias.variables['x'][:]
#    print('\t x :',x)
    y = convertedSet.createVariable('y', 'i4', ('j',))
    y.units = 'm'
    y.long_name = 'cell index along second dimension'
    y.standard_name = 'projection_x_coordinate'
    y[:] = origDrias.variables['y'][:]
#    print('\t y :',y)
    convertedSet.Conventions = origDrias.Conventions ;
    convertedSet.institution = origDrias.institution ;
    convertedSet.institute_id = origDrias.institute_id ;
    convertedSet.project_id =origDrias.project_id ;
    convertedSet.model_id = origDrias.model_id ;
    convertedSet.frequency = origDrias.frequency ;
    convertedSet.product = origDrias.product ;
    convertedSet.contact = origDrias.contact ;
    convertedSet.references = origDrias.references ;
    convertedSet.creation_date = origDrias.creation_date ;
    convertedSet.NCO = origDrias.NCO ;
    convertedSet.driving_experiment_name = origDrias.driving_experiment_name ;
    convertedSet.experiment = origDrias.experiment ;
    convertedSet.experiment_id = origDrias.experiment_id ;
    convertedSet.model = origDrias.model ;
    convertedSet.comment = origDrias.comment ;
    convertedSet.history = origDrias.history ;
    convertedSet.nco_openmp_thread_number = origDrias.nco_openmp_thread_number ;
#    for var in convertLyonTempYearMonth.variables.keys() :
#        print ('\t', var,
#               '\t', convertLyonTempYearMonth.variables[var].dimensions, 
#               '\t', convertLyonTempYearMonth.variables[var].shape, 
#               '\t', convertLyonTempYearMonth.variables[var].dtype)
    convertedSet.close()

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/rstr_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/rstr_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc 

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/rstr_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/rstr_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc 

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/rstr_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/rstr_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc 

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/rstr_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/rstr_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc 



### Température maximale

détection des variables (facultatif)

In [8]:
for name in ['tasmax_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc',
             'tasmax_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',
             'tasmax_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',
             'tasmax_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc'
            ] :
    namein = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/' + name
    nameout = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/' + name
#    print (namein, '\n',  nameout,'\n')
    origDrias = nc.Dataset(namein)
#    print('Dimensions disponibles :', origDrias.dimensions.keys(),'\n') # get all variable names
#    for var in origDrias.dimensions.keys() :#
#        print ('\t', origDrias.dimensions[var],
#               '\t', origDrias.dimensions[var].name,
#               '\t', origDrias.dimensions[var].size)
#    print('\nVariables disponibles :', origDrias.variables.keys(),'\n') # get all variable names
#    for var in origDrias.variables.keys() :
#        print ('\t', var,
#               '\t', origDrias.variables[var].dimensions, 
#               '\t', origDrias.variables[var].shape, 
#               '\t', origDrias.variables[var].dtype)

In [9]:
for name in ['tasmax_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc',
             'tasmax_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',
             'tasmax_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',
             'tasmax_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc'
            ] :
    namein = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/' + name
    nameout = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/' + name
    print (namein, '\n',  nameout,'\n')
    origDrias = nc.Dataset(namein)
    try: os.remove(nameout)  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
    except OSError : pass
    convertedSet = nc.Dataset(nameout, mode='w', format='NETCDF4') 
    for var in origDrias.dimensions.keys() :
        thisDim = origDrias.dimensions[var]
        if (thisDim.name == 'time') :
            convertedSet.createDimension(thisDim.name)
        else :
            convertedSet.createDimension(thisDim.name, thisDim.size)     # latitude axis
#
# variables detected in the original file
#
    tasmax = convertedSet.createVariable('tasmax', 'f4', ('time','j','i'),fill_value=1.e+20)
    tasmax.units = 'l'
    tasmax.long_name = 'Near-Surface Specific Humidity'
    tasmax.standard_name = 'specific_humidity'
    tasmax._CoordinateAxisType = 'lon lat'
    tasmax[:,:,:] = origDrias.variables['tasmax'][:,:,:] 
#    print('\t\t huss : ',huss)
    i = convertedSet.createVariable('i', 'f4', ('i',))
    i.long_name = 'cell index along first dimension'
    i[:] = origDrias.variables['i'][:]
#    print('\t i :',i)
#    print('\t',tMax_26_Lyon.variables['i'][:])
    j = convertedSet.createVariable('j', 'f4', ('j',))
    j.long_name = 'cell index along second dimension'
    j[:] = origDrias.variables['j'][:]
#    print('\t j :',j)
    lat = convertedSet.createVariable('lat', 'f4', ('j','i'))
    lat.units = 'degrees_north'
    lat.long_name = 'latitude'
    lat.standard_name = 'latitude'
    lat._CoordinateAxisType = 'Lat'
    lat[:,:]  = origDrias.variables['lat'][:,:] 
#    print('\t lat :',lat)
    lon = convertedSet.createVariable('lon', 'f4', ('j','i',))
    lon.units = 'degrees_east'
    lon.long_name = 'longitude'
    lon.standard_name = 'longitude'
    lon._CoordinateAxisType = 'Lon'
    lon[:,:] = origDrias.variables['lon'][:,:]
#    print('\t lon :',lon)
    time = convertedSet.createVariable('time', 'f4', ('time',))
    time.units = 'hours since 1949-12-01 00:00:00'
    time.long_name = 'time'
    time.standard_name = 'time'
    time[:] = origDrias.variables['time'][:]
#    print('\t time :',time)
    x = convertedSet.createVariable('x', 'i4', ('i',))
    x.units = 'm'
    x.long_name = 'cell index along first dimension'
    x.standard_name = 'projection_x_coordinate'
    x[:] = origDrias.variables['x'][:]
#    print('\t x :',x)
    y = convertedSet.createVariable('y', 'i4', ('j',))
    y.units = 'm'
    y.long_name = 'cell index along second dimension'
    y.standard_name = 'projection_x_coordinate'
    y[:] = origDrias.variables['y'][:]
#    print('\t y :',y)
    convertedSet.Conventions = origDrias.Conventions ;
    convertedSet.institution = origDrias.institution ;
    convertedSet.institute_id = origDrias.institute_id ;
    convertedSet.project_id =origDrias.project_id ;
    convertedSet.model_id = origDrias.model_id ;
    convertedSet.frequency = origDrias.frequency ;
    convertedSet.product = origDrias.product ;
    convertedSet.contact = origDrias.contact ;
    convertedSet.references = origDrias.references ;
    convertedSet.creation_date = origDrias.creation_date ;
    convertedSet.NCO = origDrias.NCO ;
    convertedSet.driving_experiment_name = origDrias.driving_experiment_name ;
    convertedSet.experiment = origDrias.experiment ;
    convertedSet.experiment_id = origDrias.experiment_id ;
    convertedSet.model = origDrias.model ;
    convertedSet.comment = origDrias.comment ;
    convertedSet.history = origDrias.history ;
    convertedSet.nco_openmp_thread_number = origDrias.nco_openmp_thread_number ;
#    for var in convertLyonTempYearMonth.variables.keys() :
#        print ('\t', var,
#               '\t', convertLyonTempYearMonth.variables[var].dimensions, 
#               '\t', convertLyonTempYearMonth.variables[var].shape, 
#               '\t', convertLyonTempYearMonth.variables[var].dtype)
    convertedSet.close()

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/tasmax_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/tasmax_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc 

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/tasmax_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/tasmax_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc 

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/tasmax_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/tasmax_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc 

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/tasmax_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/tasmax_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc 



### Température minimale

détection des variables (facultatif)

In [10]:
for name in ['tasmin_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc',
             'tasmin_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',
             'tasmin_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',
             'tasmin_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc'
            ] :
    namein = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/' + name
    nameout = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias/' + name
#    print (namein, '\n',  nameout,'\n')
    origDrias = nc.Dataset(namein)
#    print('Dimensions disponibles :', origDrias.dimensions.keys(),'\n') # get all variable names
#    for var in origDrias.dimensions.keys() :
#        print ('\t', origDrias.dimensions[var],
#               '\t', origDrias.dimensions[var].name,
#               '\t', origDrias.dimensions[var].size)
#    print('\nVariables disponibles :', origDrias.variables.keys(),'\n') # get all variable names
#    for var in origDrias.variables.keys() :
#        print ('\t', var,
#               '\t', origDrias.variables[var].dimensions, 
#               '\t', origDrias.variables[var].shape, 
#               '\t', origDrias.variables[var].dtype)

In [11]:
for name in ['tasmin_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc',
             'tasmin_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc',
             'tasmin_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc',
             'tasmin_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc'
            ] :
    namein = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/' + name
    nameout = '/home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDrias' + name
    print (namein, '\n',  nameout,'\n')
    origDrias = nc.Dataset(namein)
    try: os.remove(nameout)  # par sécurité efface le fichier portant ce nom ! attention aux pertes possibles
    except OSError : pass
    convertedSet = nc.Dataset(nameout, mode='w', format='NETCDF4') 
    for var in origDrias.dimensions.keys() :
        thisDim = origDrias.dimensions[var]
        if (thisDim.name == 'time') :
            convertedSet.createDimension(thisDim.name)
        else :
            convertedSet.createDimension(thisDim.name, thisDim.size)     # latitude axis
#
# variables detected in the original file
#
    tasmin = convertedSet.createVariable('tasmin', 'f4', ('time','j','i'),fill_value=1.e+20)
    tasmin.units = 'l'
    tasmin.long_name = 'Near-Surface Specific Humidity'
    tasmin.standard_name = 'specific_humidity'
    tasmin._CoordinateAxisType = 'lon lat'
    tasmin[:,:,:] = origDrias.variables['tasmin'][:,:,:] 
#    print('\t\t huss : ',huss)
    i = convertedSet.createVariable('i', 'f4', ('i',))
    i.long_name = 'cell index along first dimension'
    i[:] = origDrias.variables['i'][:]
#    print('\t i :',i)
#    print('\t',tMax_26_Lyon.variables['i'][:])
    j = convertedSet.createVariable('j', 'f4', ('j',))
    j.long_name = 'cell index along second dimension'
    j[:] = origDrias.variables['j'][:]
#    print('\t j :',j)
    lat = convertedSet.createVariable('lat', 'f4', ('j','i'))
    lat.units = 'degrees_north'
    lat.long_name = 'latitude'
    lat.standard_name = 'latitude'
    lat._CoordinateAxisType = 'Lat'
    lat[:,:]  = origDrias.variables['lat'][:,:] 
#    print('\t lat :',lat)
    lon = convertedSet.createVariable('lon', 'f4', ('j','i',))
    lon.units = 'degrees_east'
    lon.long_name = 'longitude'
    lon.standard_name = 'longitude'
    lon._CoordinateAxisType = 'Lon'
    lon[:,:] = origDrias.variables['lon'][:,:]
#    print('\t lon :',lon)
    time = convertedSet.createVariable('time', 'f4', ('time',))
    time.units = 'hours since 1949-12-01 00:00:00'
    time.long_name = 'time'
    time.standard_name = 'time'
    time[:] = origDrias.variables['time'][:]
#    print('\t time :',time)
    x = convertedSet.createVariable('x', 'i4', ('i',))
    x.units = 'm'
    x.long_name = 'cell index along first dimension'
    x.standard_name = 'projection_x_coordinate'
    x[:] = origDrias.variables['x'][:]
#    print('\t x :',x)
    y = convertedSet.createVariable('y', 'i4', ('j',))
    y.units = 'm'
    y.long_name = 'cell index along second dimension'
    y.standard_name = 'projection_x_coordinate'
    y[:] = origDrias.variables['y'][:]
#    print('\t y :',y)
    convertedSet.Conventions = origDrias.Conventions ;
    convertedSet.institution = origDrias.institution ;
    convertedSet.institute_id = origDrias.institute_id ;
    convertedSet.project_id =origDrias.project_id ;
    convertedSet.model_id = origDrias.model_id ;
    convertedSet.frequency = origDrias.frequency ;
    convertedSet.product = origDrias.product ;
    convertedSet.contact = origDrias.contact ;
    convertedSet.references = origDrias.references ;
    convertedSet.creation_date = origDrias.creation_date ;
    convertedSet.NCO = origDrias.NCO ;
    convertedSet.driving_experiment_name = origDrias.driving_experiment_name ;
    convertedSet.experiment = origDrias.experiment ;
    convertedSet.experiment_id = origDrias.experiment_id ;
    convertedSet.model = origDrias.model ;
    convertedSet.comment = origDrias.comment ;
    convertedSet.history = origDrias.history ;
    convertedSet.nco_openmp_thread_number = origDrias.nco_openmp_thread_number ;
#    for var in convertLyonTempYearMonth.variables.keys() :
#        print ('\t', var,
#               '\t', convertLyonTempYearMonth.variables[var].dimensions, 
#               '\t', convertLyonTempYearMonth.variables[var].shape, 
#               '\t', convertLyonTempYearMonth.variables[var].dtype)
    convertedSet.close()

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/tasmin_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDriastasmin_metro_CNRM_Aladin_histo_QT_REF_19500101-20051231.nc 

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/tasmin_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDriastasmin_metro_CNRM_Aladin_rcp2.6_QT_RCP2.6_20060101-21001231.nc 

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/tasmin_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDriastasmin_metro_CNRM_Aladin_rcp4.5_QT_RCP4.5_20060101-21001231.nc 

/home/vidal/TremplinDesSciences/2019/ClimatLyon/DataDrias/tasmin_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc 
 /home/vidal/TremplinDesSciences/2019/ClimatLyon/ConvertedDriastasmin_metro_CNRM_Aladin_rcp8.5_QT_RCP8.5_20060101-21001231.nc 

